# LEET CODE
## Problem-1164: Product Price at a Given Date
Class: Medium

Source: https://leetcode.com/problems/product-price-at-a-given-date

## Description
Table: Products
| Column Name   | Type    |
|---------------|---------|
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |

- (product_id, change_date) is the primary key of this table.
- Each row of this table indicates that the price of some product was changed to a new price at some date.

## To Do
Write an SQL query to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The query result format is in the following example.

## Example

Input: 

Products table:
| product_id | new_price | change_date |
|------------|-----------|-------------|
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |

Output: 
| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |

## Importing

In [1]:
import pandas as pd
from pandasql import sqldf

## Define Schema

In [3]:
data = {
    "product_id": [1, 2, 1, 1, 2, 3],
    "new_price": [20, 50, 30, 35, 65, 20],
    "change_date": ["2019-08-14", "2019-08-14", "2019-08-15", "2019-08-16", "2019-08-17", "2019-08-18"]
}

Products = pd.DataFrame(data)
Products["change_date"] = pd.to_datetime(Products["change_date"])

Products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


## Task

In [20]:
# Define the SQL query
query = """
WITH id AS (
    SELECT DISTINCT(product_id) FROM Products
),
last_change AS (
    SELECT product_id, new_price AS price
    FROM Products
    WHERE change_date < '2019-08-17'
    GROUP BY product_id
    HAVING MAX(change_date)
)
SELECT product_id, COALESCE(price, 10) AS price
FROM id
LEFT JOIN last_change USING(product_id)
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Products':Products})

# Display the result dataframe
display(result)

,product_id,price
0,1,35
1,2,50
2,3,10


In [36]:
# Define the SQL query
query = """
WITH last_price AS (
    SELECT 
        DISTINCT product_id,
        FIRST_VALUE(new_price) OVER (PARTITION BY product_id ORDER BY change_date DESC) AS price
    FROM Products
    WHERE DATE(change_date) <= '2019-08-16'
),
id AS (
    SELECT DISTINCT(product_id) FROM Products
)
SELECT product_id, COALESCE(price, 10) AS price
FROM id
LEFT JOIN last_price USING(product_id)
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Products':Products})

# Display the result dataframe
display(result)

,product_id,price
0,1,35
1,2,50
2,3,10
